# Echochambers Episode 1: The Optimization Menace

Created by Kristof Gazso (@kristofgazso)

Echochambers are the usually most vilifying and useless form of communities. Little critical thinking is invested into the ideas circulated beyond the justification of already held beliefs, and any conflicting ideas are usually ridiculed or just ignored at worst. Platforms like Twitter and Reddit provide safe haven for a many echochambers on the internet as they make it incredibly easy to become intrenched in certain communities (like subreddits or twitter sub-cultures).

Here I attempt to model some of the behaviours of echochambers: simply at first, then building up complexity. I will see if I can find some non-intuitive and interesting conclusions while doing so.

A lot of the initial modelling will come from Frank Witte's ECON0055 Economics of Science lectures

NOTE: the models created here are just that: models. There is a lot of oversimplification that happens when creating them and they shouldn't be treated as 100% accurate in predicting human behaviour.

To run the cells, simply press `SHIFT+ENTER` on them. Feel free to play around with any initial parameters in the cells and rerun them.

In [ ]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import matplotlib
from IPython import display
import networkx as nx

In [ ]:
# We are going to define some functions that will allow us to repesent the data that we come across during modelling

def plot_beliefs_and_evidence(beliefs, evidence):
    fig, axs = plt.subplots(2)
    axs[0].imshow(beliefs.detach(), cmap="bwr", norm=matplotlib.colors.Normalize(-1, 1, True))
    axs[0].set_title("Agent beliefs", fontsize="x-large")
    axs[0].set_xlabel("Agent number")
    axs[0].set_xticks(range(beliefs.shape[1]))
    axs[0].tick_params(axis="y", which="both", left=False, right=False, labelleft=False)
    axs[1].imshow(evidence.detach(), cmap="bwr", norm=matplotlib.colors.Normalize(-1, 1, True))
    axs[1].set_title("Agent evidence", fontsize="x-large")
    axs[1].set_xlabel("Agent number")
    axs[1].set_xticks(range(evidence.shape[1]))
    axs[1].tick_params(axis="y", which="both", left=False, right=False, labelleft=False)
    plt.show()

def show_adjacency_matrix(listen_factor):
    fig, ax = plt.subplots()
    ax.imshow(listen_factor.detach(), cmap="PRGn", norm=matplotlib.colors.Normalize(-1, 1, True))
    ax.set_title("Agent beliefs", fontsize="x-large")
    ax.set_xlabel("Listener")
    ax.set_ylabel("Listening")
    _ = ax.set_xticks(torch.arange(0, listen_factor.shape[1], listen_factor.shape[1]//10))
    _ = ax.set_yticks(torch.arange(0, listen_factor.shape[0], listen_factor.shape[1]//10))
    
def show_adjacency_graph(listen_factor):
    G2 = nx.from_numpy_array(listen_factor.numpy())
    _ = nx.draw_circular(G2, arrows=True)
    _ = plt.axis('equal')

In [ ]:
learning_rate = 1e-4
num_agents = 10
scepticism = torch.Tensor([2]) # form 0 to inf, the higher the number the more sceptic they are

beliefs = torch.divide(torch.randn(1, num_agents), 10).requires_grad_()
evidence = torch.divide(torch.randn(1, num_agents), 10)

Initially, we will model agents in a relatively simple way. Each of the ten agents will have a fixed evidence variable, which will be relatively small random value (0 mean and 0.1 SD). Then, they will have another attribute called their belief, which is another small random value (0 mean and 0.1 SD) that represents which way on a particular issue they believe in. In this simulation, the issue will just have two opposing viewpoints, and will be a continuous number from -inf to +inf representing which side and how strong they believe in the viewpoint.

In [ ]:
print("Initial beliefs and evidence:")
plot_beliefs_and_evidence(beliefs, evidence)

Finally, we get them to maximize their utility. Their utility will be defined simply as the product of their beliefs and the evidence for that belief, however we will subtract from this a value that is proportional to their belief to show that they might have some scepticism. It makes sense that for them to maximize their utility, they will have to change their beliefs to fit the little evidence that they have by making it the same sign. After each "timeslot", they will look at which way they should change their beliefs to increase their utility and make a small step towards it.

In [ ]:
for t in range(50000):

    if t % 10000 == 9999:
        display.clear_output(wait=True)
        print(t, "steps")
        plot_beliefs_and_evidence(beliefs, evidence)
        print("Average agent utility:", utility.mean().item())

    utility = beliefs * evidence - scepticism/2*beliefs.square()
    
    # Find gradients for the beliefs to increase utility
    utility.backward(torch.ones_like(utility))

    # Update weights using gradient descent
    with torch.no_grad():
        beliefs += learning_rate * beliefs.grad
        beliefs.grad.zero_()


As we can see, they change their beliefs in order to increase their utility, settling when their beliefs are close to their evidence

# Listening to Peer beliefs

So far, in order to decide on their beliefs, agents only had their own evidence to work with. This is too simplistic. In real life, a lot of our beliefs are influenced by the people we listen to and interact with, and so I will attempt to model this interaction. In this model, each agent will be allocated a fixed 0 or 1 pointing to every other agent to signify if they "listen" to this other agent. If they do, then part of their utility will be derived from having similar beliefs to them. This will be a directed graph, i.e. just because agent X listens to agent Y, it doesn't mean that agent Y will necessarily listen to agent X.

In [ ]:
learning_rate = 1e-4
num_agents = 10
scepticism = torch.Tensor([2]) # form 0 to inf, the higher the number the more sceptic they are
peer_multiplier = torch.Tensor([1]) # how much agents value the people they listen to
listen_chance = 0.35 # the chance of agent x listening to agent y, the higher the number the more connections

beliefs = torch.divide(torch.randn(1, num_agents), 10).requires_grad_()
evidence = torch.divide(torch.randn(1, num_agents), 10)
listen_factor = (torch.rand(size=(num_agents,num_agents)) < listen_chance).int() # generate either 0 or 1
listen_factor = listen_factor.fill_diagonal_(0) # they will listen to themselves, but that is already included

show_adjacency_matrix(listen_factor)

On the adjacency matrix above, the x axis represenets if the agent is "listening", and the y axis represents if that agent is being "listened to". If (4, 2) is green for example, it would mean that 2 listens to 4.

In [ ]:
show_adjacency_graph(listen_factor)

Above might be a clearer representation of who listens to who

In [ ]:
for t in range(20000):

    if t % 1000 == 999:
        display.clear_output(wait=True)
        print(t, "steps")
        plot_beliefs_and_evidence(beliefs, evidence)
        print("Average agent utility:", utility.mean().item())
        
    self_utility = beliefs * evidence
    scepticism_utility = scepticism/2*beliefs.square()
    peer_utility = peer_multiplier * torch.sum((listen_factor * beliefs.t()), 0) * beliefs

    utility = self_utility - scepticism_utility + peer_utility
    
    # Find gradients for the beliefs to increase utility
    utility.backward(torch.ones_like(utility))

    # Update weights using gradient descent
    with torch.no_grad():
        beliefs += learning_rate * beliefs.grad
        beliefs.grad.zero_()


Its wroth resetting and rerunning this simulation several times. You will find most of the times that all of the agents end up having the same beliefs, but it is quite random whether that will be red (positive) or blue (negative).

# Changing avaiable evidence

What happens if we also let agents "change" the evidence that they have available for themselves. This could represent selectively accepting or rejecting evidence. In order to make it so that this effect doesn't make the model explode too quickly, we also include an `evidence_pressure` parameter that quadratically punishes agents for having their subjective evidence being far away from the real evidence.

In [ ]:
beliefs = torch.divide(torch.randn(1, num_agents), 10).requires_grad_()
evidence = torch.divide(torch.randn(1, num_agents), 10).requires_grad_()
listen_chance = 0.35 # the chance of agent x listening to agent y, the higher the number the more connections
listen_factor = (torch.rand(size=(num_agents,num_agents)) < listen_chance).int() # generate either 0 or 1
listen_factor = listen_factor.fill_diagonal_(0) # they will listen to themselves, but that doesn't need to be part of the listen_factor

real_evidence = 0
evidence_pressure = torch.Tensor([4])

In [ ]:
show_adjacency_matrix(listen_factor)

In [ ]:
show_adjacency_graph(listen_factor)

In [ ]:
learning_rate = 1e-3
for t in range(2000):
    if t % 100 == 99:
        display.clear_output(wait=True)
        print(t, "steps")
        plot_beliefs_and_evidence(beliefs, evidence)
        print("Average agent utility:", utility.mean().item())

    utility = beliefs * evidence - scepticism/2*beliefs.square() + peer_multiplier * torch.sum((listen_factor * beliefs.t()), 0) * beliefs - evidence_pressure * (evidence - real_evidence).square()
    
    # Find gradients for the beliefs to increase utility
    utility.backward(torch.ones_like(utility))

    # Update weights using gradient descent
    with torch.no_grad():
        beliefs += learning_rate * beliefs.grad
        evidence += learning_rate * evidence.grad
        beliefs.grad.zero_()
        evidence.grad.zero_()


Interestingly we see that agents usually end up completely radicalized in their beliefs before the "evidence" that they have starts to drastically change.

# Allowing them to change who they listen to

Until now we made it so that the agents an agent listened to was fixed and unchangeable. This is quite an unrealistic scenario as many people would choose to listen more to people to they agree with and less to people they disagree with. Furthermore, we can allow agents to have negative `listen_factor` values, which would mean that they would believe the opposite of what another agent believes. We will start out with a small and normally distributed `listen_factor`.

NOTE: for this model we have disabled the ability for agents to change their evidence. We will reenable this in the next model.

In [ ]:
num_agents = 10
scepticism = torch.Tensor([2]) # form 0 to inf, the higher the number the more sceptic they are
peer_multiplier = torch.Tensor([2.5]) # how much agents value the people they listen to

In [ ]:
beliefs = torch.divide(torch.randn(1, num_agents), 10).requires_grad_()
evidence = torch.divide(torch.randn(1, num_agents), 50)
listen_factor = torch.randn((num_agents, num_agents)).divide(10).fill_diagonal_(0).requires_grad_() # generate either 0 or 1

In [ ]:
show_adjacency_matrix(listen_factor)

In [ ]:
learning_rate = 1e-3
for t in range(15000):
    if t % 1000 == 999:
        display.clear_output(wait=True)
        print(t, "steps")
        plot_beliefs_and_evidence(beliefs, evidence)
        show_adjacency_matrix(listen_factor)
        print("Average agent utility:", utility.mean().item())

    self_utility = beliefs * evidence
    scepticism_utility = scepticism/2*beliefs.square()
    peer_utility = peer_multiplier * torch.sum((listen_factor * beliefs.t()), 0) * beliefs
    utility =  self_utility - scepticism_utility + peer_utility
    
    # Find gradients for the beliefs to increase utility
    utility.backward(torch.ones_like(utility))

    # Update weights using gradient descent
    with torch.no_grad():
        beliefs += learning_rate * beliefs.grad
        beliefs.grad.zero_()
        beliefs = beliefs.clamp_(-1, 1)
        
        listen_factor += learning_rate * listen_factor.grad
        listen_factor.grad.zero_()
        listen_factor = listen_factor.fill_diagonal_(0)
        listen_factor = listen_factor.clamp_(-1, 1)


We see that agents have developed a complete echochamber of ideas. They end up with a very peculiar `listen_factor`. They end up listening completely to agents that have the same beliefs as them and end up wanting to believe the opposite of agents that have the opposite beliefs as them. Basically, in this model, they usually end up forming two 'bubbles' where they listen completely to agents that are in the same bubbles and want to disagree with agents who are in the opposite bubble.

Also worth noting that agents seem to decide their beliefs slightly before they make major changes to who they listen to.

# Allowing them to selectively include new evidence

In this final model of Echochambers Episode 1, we simply allow the agents to change their evidence again.

In [ ]:
learning_rate = 1e-3
num_agents = 10
scepticism = torch.Tensor([2]) # form 0 to inf, the higher the number the more sceptic they are
peer_multiplier = torch.Tensor([2.5]) # how much agents value the people they listen to
evidence_pressure = torch.Tensor([2]) # lets make this have quite a harsh penalty

beliefs = torch.divide(torch.randn(1, num_agents), 10).requires_grad_()
evidence = torch.divide(torch.randn(1, num_agents), 50).requires_grad_()
listen_factor = torch.randn((num_agents, num_agents)).divide(10).fill_diagonal_(0).requires_grad_() # generate either 0 or 1

real_evidence = 0 # the actual position of the evidence

show_adjacency_matrix(listen_factor)

In [ ]:
learning_rate = 1e-3
for t in range(10000):
    if t % 1000 == 999:
        #real_evidence -= 0.02
        display.clear_output(wait=True)
        print(t, "steps")
        plot_beliefs_and_evidence(beliefs, evidence)
        show_adjacency_matrix(listen_factor)
        print("Average agent utility:", utility.mean().item())

    #with torch.autograd.detect_anomaly():
    self_utility = beliefs * evidence
    scepticism_utility = scepticism/2*beliefs.square()
    peer_utility = peer_multiplier * torch.sum((listen_factor * beliefs.t()), 0) * beliefs
    evidence_scepticism_utility = evidence_pressure * (evidence - real_evidence).square()
    utility =  self_utility - scepticism_utility + peer_utility - evidence_scepticism_utility
    
    # Find gradients for the beliefs to increase utility
    utility.backward(torch.ones_like(utility))

    # Update weights using gradient descent
    with torch.no_grad():
        beliefs += learning_rate * beliefs.grad
        beliefs.grad.zero_()
        beliefs = beliefs.clamp_(-1, 1)
        
        evidence += learning_rate * evidence.grad
        evidence.grad.zero_()
        evidence = evidence.clamp_(-1, 1)
        
        listen_factor += learning_rate * listen_factor.grad
        listen_factor.grad.zero_()
        listen_factor = listen_factor.fill_diagonal_(0)
        listen_factor = listen_factor.clamp_(-1, 1)

We find that agents end up forming their echochambers a lot faster when they are capable of changing their evidence.

# Next Episode

In the next episode we will create models to see what ways there are of mitigating the formation of echochambers and how it could be possible to break agents out of them